<a href="https://colab.research.google.com/github/franklinzhou-ncsu/ST_554/blob/main/%5CTask1%5CST_554_Project_1_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ST 554 Project 1 - Task 1

Author: Franklin Zhou

Date: Feb.7.2026

Summary of the task:

This task involves writing two grid search algorithms and two gradient descent type algorithms to find the optimal constant to use for squared error loss (ends up being the sample mean) and to find the optimal intercept and slope from a simple linear regression model.



## Preparation work

### 1. Install required library

In [ ]:
!pip install ucimlrepo

### 2. Read data and data cleaning

We need to remove any observations where the C6H6(GT) or CO(GT) are -200.

In [ ]:
import ucimlrepo as uci
air_quality = uci.fetch_ucirepo(id=360)
df = air_quality.data.features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Remove any observations where the C6H6(GT) or CO(GT) are -200
df_clean = df[(df["C6H6(GT)"] != -200) & (df["CO(GT)"] != -200)].copy()

df_clean

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


## Method 1: Using a Grid Search Algorithm



### Method 1.1: Grid search on $y$ variable only

The logic behind this method:

Generate a list of different $c$ values, then compute the root mean squared error by:

$$
RMSE_{j} = \sqrt{\frac{\sum_{i=1}^n(y_i-c_j)^2}{n}}
$$

We choose the $c$ that has the minimum $RMSE$ value, that $c$ is the best guess of expected value of $y$. Theoretically, the expected value of $y$ is $\hat{y}$.

The function is below. It has 4 parameters:

- `y` is the 1D data to be analyzed
- `start` is the lower quantile of the data
- `end` is the upper quantile of the data
- `grid_zise` is the size of grid between `start` and `end`

By dafault, the `start` is set to Q1, which is 25th percentile, `end` is set to Q3, which is 75th percentile, and `grid_zise` is set to 100.


In [ ]:
def grid_search_only_y (y, start:float = 0.25, end:float = 0.75, grid_size:int = 100):
    """
    Use Grid Search for best c.
    Default quantiles are 0.25 and 0.75.
    Default grid size is 100.
    """
    # find q1 and q3 values of y
    q1 = y.quantile(start)
    q3 = y.quantile(end)

    # create a grid of values between Q1 and Q3
    grid = np.linspace(q1, q3, grid_size)

    # root mean squared error function
    def rmse_func (y, c):
        resid = y - c
        rmse = np.sqrt(np.sum(resid**2)/len(y))
        return rmse

    # create a list to save the c and RMSE values
    rmses = []

    # loop over grid to calculate RMSEs and add them to the list
    for c in grid:
        rmse = rmse_func(y, c)
        rmses.append((c, rmse))

    # determine the best c with the lowest RMSE value
    best_c, min_rmse = min(rmses, key = lambda x: x[1])

    # return minimum RMSE with its c
    return min_rmse, best_c

 Try to run the function using `C6H6(GT)` as $y$ with default setting parameters:

In [ ]:
best_guess = grid_search_only_y(df_clean['C6H6(GT)'])

print("The minimum RMSE is", best_guess[0], "with y equals to", best_guess[1])

The minimum RMSE is 7.440564471926773 with y equals to 10.282828282828284


We can check the average value of `C6H6(GT)` in the data set to see if 10.28 is close to it:

In [ ]:
np.mean(df_clean['C6H6(GT)'])

np.float64(10.275735294117647)

The best $c$ is quite close to the average of $y$. Now let's run the function using `PT08.S1(CO)` as $y$ variable with customized parameters:

In [ ]:
# run the function using PT08.S1(CO) with grid size = 1000
best_guess = grid_search_only_y(df_clean['PT08.S1(CO)'], grid_size = 1000)

print("The minimum RMSE is", best_guess[0], "with y equals to", best_guess[1])

The minimum RMSE is 218.6664431268348 with y equals to 1110.5645645645645


Check the average value of `PT08.S1(CO)`:

In [ ]:
np.mean(df_clean['PT08.S1(CO)'])

np.float64(1110.5807461873637)

The best $c$ value is quite close to the acutal average of $y$ as well. **So the `grid_search_only_y` function is valid.**

### Method 1.2: Using $y$ and another numeric variable $x$

The logic of this method is similar to the first one.

Suppose we have a linear model

$$
y = \beta_0 + \beta_1 \times x
$$

Then we generate a list of different $b_0$ and $b_1$ values, then compute the root mean squared error by:

$$
RMSE = \sqrt{\frac{\sum_{i=1}^n(y_i-\hat{y}_i)^2}{n}}
$$

where $\hat{y}_i = b_0 + b_1 \times x$.

We choose the value of $b_0$ and $b_1$ such that it has the minimum $RMSE$ value, that $b_0$ and $b_1$ are the best guess of $\beta_0$ and $\beta_1$.


The function is below. It has 8 parameters:

- `y` is the 1D data as response variable
- `x` is the 1D data as predictor variable
- `b_0_start` is the minimun of $b_0$ grid
- `b_0_end` is maximum of $b_0$ grid
- `b_0_step` is the incremental unit of $b_0$ grid
- `b_1_start` is the minimun of $b_1$ grid
- `b_1_end` is maximum of $b_1$ grid
- `b_1_step` is the incremental unit of $b_1$ grid

In [ ]:
def grid_search_y_x (y, x, b_0_start:float = -25, b_0_end:float = -15, b_0_step:float = 0.1, b_1_start:float = -5, b_1_end:float = 5, b_1_step:float = 0.01):
    """
    Grid Search using simple linear regression:
    y = b0 + b1 * x
    """
    # populate a grid of b_0 and b_1 values
    b_0_vals = np.arange(b_0_start, b_0_end + b_0_step, b_0_step)
    b_1_vals = np.arange(b_1_start, b_1_end + b_1_step, b_1_step)

    # set up default values for lowest rmse, best b_0 and b_1
    min_rmse = None
    best_b_0 = None
    best_b_1 = None

    # root mean squared error function
    def rmse_func (y, x, b_0, b_1):
        pred_y = b_0 + b_1 * x # calculate predicted y
        resid = y - pred_y # calculate residual
        rmse = np.sqrt(np.sum(resid**2)/len(y)) # calculate the rmse
        return rmse

    # Iterate through grid
    for b_0 in b_0_vals:
        for b_1 in b_1_vals:
            rmse = rmse_func(y, x, b_0, b_1) # call the rmse function
            # keep the lowest rmse value and b_0, b_1
            if min_rmse is None or rmse < min_rmse:
                min_rmse = rmse
                best_b_0 = b_0
                best_b_1 = b_1

    return best_b_0, best_b_1, min_rmse

Try the function by using `PT08.S1(CO)` as $x$ variable and `C6H6(GT)` as $y$ variable:

In [ ]:
best_fit = grid_search_y_x(y = df_clean['C6H6(GT)'], x = df_clean['PT08.S1(CO)'])
intercept = best_fit[0]
slope = best_fit[1]
rmse = best_fit[2]
print("The intercept of the best fit SLM is ", intercept, "the slope of the best fit SLM is ", slope, "the RMSE value is ", rmse)

The intercept of the best fit SLM is  -22.99999999999997 the slope of the best fit SLM is  0.02999999999989278 the RMSE value is  3.5429053908306782


The best-fit $b_0$ of OLS model is $-22.95575$ while the best-fit $b_1$ of OLS model is $0.02992$, which is quite close to the best-fit values in this case.
**So, the `grid_search_y_x` function is valid.**

We can use the `best_fit` result to make prediction:

In [ ]:
# if PT08.S1(CO) = 946, 1075, and 1246:
new_x = [946, 1075, 1246]
preds = [intercept + slope * i for i in new_x]
preds

[np.float64(5.3799999998985975),
 np.float64(9.249999999884764),
 np.float64(14.379999999866428)]

- When `PT08.S1(CO)` = 946, the predicted `C6H6(GT)` value is 5.38
- When `PT08.S1(CO)` = 1075, the predicted `C6H6(GT)` value is 9.25
- When `PT08.S1(CO)` = 1246, the predicted `C6H6(GT)` value is 14.38

## Method 2: Using a Gradient Descent Algorithm

### Method 2.1: Gradient Descent only on $y$

Basic logic of this method:

- Start with an initial guess far away from $\bar{y}$.

- Update the guess based on an approximation to the derivative (slope) of the RMSE and a “step-size”(generally some small value):

$$\text{new_value} = \text{old_value} - \text{(slope)} * \text{step_size}$$

- Check if the movement ($|\text{new_value} - \text{old_value}|$) exceeds some small numeric tolerance. If so, we move quite a bit and repeat the steps above until the $|\text{new_value} - \text{old_value}|$ is below our tolerance limit.

The slope can be approximated by quotient difference:

$$
\text{diff_quotient} = \frac{RMSE(c+\Delta) - RMSE(c)}{\Delta}
$$

The function is below. It has 6 parameters:

- `y` is the 1D data to be analyzed

- `start` is the initial guess

- `delta` is a small change in difference quotient function

- `step_size` is the update of a small moving distance

- `num_tol` is the value of tolerance

- `max_iter` is the maximum number of iterations

In [ ]:
def gradient_descent_only_y (y, start:float = 0, delta:float = 0.001, step_size:float = 0.01, num_tol:float = 0.0001, max_iter:int = 10000):
    """
    Gradient Descent for y
    Default delta is 0.001
    Default step size is 0.01
    Default tolerance is 0.0001
    Default maximum number of iterations is 10000
    """
    # initiate c to the start value of y
    cur_c = start

    # root mean squared error function
    def rmse_func (y, cur_c):
        resid = y - cur_c
        rmse = np.sqrt(np.sum(resid ** 2)/len(y))
        return rmse

    # function to calculate the quotient difference between rmse(c + delta) and rmse(c)
    def diff_quotient_func(y, cur_c, delta):
        diff_quo = (rmse_func(y, (cur_c + delta)) - rmse_func(y, cur_c)) / delta # call the calculate_rmse() function
        return diff_quo

    # iterate
    for i in range(max_iter):
        # evaluate the difference quotient at cur_c.
        diff_quotient = diff_quotient_func(y, cur_c, delta)
        # update c
        new_c = cur_c - diff_quotient * step_size
        # check tolerance
        if abs(new_c - cur_c) < num_tol:
            return new_c, i # if condition met, output current c and iteration
        else:
            cur_c = new_c

    return cur_c, max_iter

In [ ]:
gradient_descent_only_y(df_clean['C6H6(GT)'])

(np.float64(10.200993520073336), 3965)

The output shows that the optimal prediction of `C6H6(GT)` is 10.2, which is very close to the true average value. Try the function on `PT08.S1(CO)` variable:

In [ ]:
pred = gradient_descent_only_y(df_clean['PT08.S1(CO)'], step_size = 0.1, start = 1100)
pred[0]

np.float64(1110.3616956683077)

The output shows that the optimal prediction of `PT08.S1(CO)` is 1110.36, which is also close to the true average value.

**So, the `gradient_descent_only_y` function is a valid one.**

### Method 2.2: Gradient Descent using $y$ and another numeric variable $x$

The logic of this method is similar to the previous one. Now we have two variables to optimize over: $b_0$ and $b_1$.

We need to update $b_0$ and $b_1$ by using the same equation:

$$
\text{new_b}_i = \text{current_b}_i - \text{diff_quotient_b}_i \times \text{step_size_b}_i
$$

where the $\text{diff_quotient_b}_i$ can be calculated by:

$$
\text{diff_quotient_b}_0 = \frac{RMSE(b_0 + \Delta_0, b_1) - RMSE(b_0,b_1)}{\Delta_0}
$$

and

$$
\text{diff_quotient_b}_1 = \frac{RMSE(b_0, b_1 + \Delta_1) - RMSE(b_0,b_1)}{\Delta_1}
$$

Then we can find the euclidean distance between vector $(\text{current_b}_0, \text{current_b}_1)$ and $(\text{new_b}_0, \text{new_b}_1)$:

$$
distance = \sqrt{(\text{new_b}_0 - \text{current_b}_0)^2 + (\text{new_b}_1 - \text{current_b}_1)^2}
$$

If the distance is greater than some small tolerance, we update the $\text{current_b}_0$ and $\text{current_b}_1$ to be the $\text{new_b}_0$ and $\text{new_b}_1$ values and continue the iteration untill the distance is less than the tolerance.


In [ ]:
def gradient_descent_y_x (y, x, b_0_start:float = -20, b_1_start:float = 0, step_size_b0:float = 0.5, step_size_b1:float = 0.00005, delta_b_0:float = 0.005, delta_b_1:float = 0.005, num_tol:float = 0.0001, max_iter:int = 100000):
    """
    Gradient Descent for simple linear regression
    y = b0 + b1 * x
    Default starting value of b_0 is -20,
    Default starting value of b_1 is 0,
    Default step increase of b_0 is 0.5,
    Default step increase of b_1 is 0.00005,
    Default delta of b_0 is 0.005,
    Default delta of b_1 is 0.005,
    Default tolerance is 0.0001,
    Default maximum number of iterations is 100000
    """
    cur_b0 = b_0_start
    cur_b1 = b_1_start

    # root mean squared error function
    def rmse_func (y, x, b0, b1):
        pred_y = b0 + b1 * x # calculate predicted y
        resid = y - pred_y # calculate residual
        rmse = np.sqrt(np.sum(resid**2)/len(y)) # calculate the rmse
        return rmse

    # difference quotient function for b0
    def diff_quo_b0_func (y, x, b_0, b_1, delta_b_0):
        diff_quo_b0 = (rmse_func(y, x, b0 = (b_0 + delta_b_0), b1 = b_1) - rmse_func(y, x, b0 = b_0, b1 = b_1)) / delta_b_0 # call rmse_func() function
        return diff_quo_b0

    # difference quotient function for b1
    def diff_quo_b1_func (y, x, b_0, b_1, delta_b_1):
        diff_quo_b1 = (rmse_func(y, x, b0 = b_0, b1 = (b_1 + delta_b_1)) - rmse_func(y, x, b0 = b_0, b1 = b_1)) / delta_b_1 # call rmse_func() function
        return diff_quo_b1

    for i in range(max_iter):

        # evaluate the difference quotient for b_0 at the cur_b0 and cur_b1 values
        diff_quotient_b0 = diff_quo_b0_func(y, x, b_0 = cur_b0, b_1 = cur_b1, delta_b_0 = delta_b_0)
        # update b_0
        new_b0 = cur_b0 - diff_quotient_b0 * step_size_b0

        # evaluate the difference quotient for b_1 at the new_b0 and cur_b1 values.
        diff_quotient_b1 = diff_quo_b1_func(y, x, b_0 = new_b0, b_1 = cur_b1, delta_b_1 = delta_b_1)
        # update b1
        new_b1 = cur_b1 - diff_quotient_b1 * step_size_b1

        # calculate the Euclidean distance
        dist = np.sqrt((new_b0 - cur_b0)**2 + (new_b1 - cur_b1)**2)

        # check tolerance and update cur_b0, cur_b1 if need
        if dist < num_tol:
            return new_b0, new_b1, i # if condition met, output current b0, b1 and iteration number
        else:
            cur_b0 = new_b0
            cur_b1 = new_b1

        ### uncomment to track the converging status ###
        #if i % 5000 == 0:  # print every 5000 iterations
        #    print(f"Iteration {i}: b_0={cur_b0:.4f}, b_1={cur_b1:.6f}, dist={dist:.6f}")

    return cur_b0, cur_b1, max_iter

Now try to run the function by using `PT08.S1(CO)` as  $x$  variable and `C6H6(GT)` as $y$ variable:

In [ ]:
best_fit = gradient_descent_y_x(y = df_clean['C6H6(GT)'], x = df_clean['PT08.S1(CO)'])
intercept = best_fit[0]
slope = best_fit[1]

In [ ]:
print("The intercept of the best approximate SLM is ", intercept, "the slope of the best approximate SLM is ", slope)

The intercept of the best fit SLM is  -22.318218464984767 the slope of the best fit SLM is  -0.001448260517237225


Then we can use the best approximate model to make predictions:

In [ ]:
new_x = [946, 1075, 1246]
preds = [intercept + slope * i for i in new_x]
preds

[np.float64(-23.714716638595412),
 np.float64(-23.90697394048833),
 np.float64(-24.16182664299755)]

- When PT08.S1(CO) = 946, the predicted C6H6(GT) value is -23.71
- When PT08.S1(CO) = 1075, the predicted C6H6(GT) value is -23.90
- When PT08.S1(CO) = 1246, the predicted C6H6(GT) value is -24.16

We can see that the `gradient_descent_y_x` function does not perform well as we expected. Maybe it's due to bad parameter tuning. We still have much space to improve it.

## References

- Return evenly spaced numbers over a specified interval by using `numpy.linspace()` function: https://numpy.org/doc/stable/reference/generated/numpy.linspace.html

- Return the sublist that has the minimum value of the second element in a main list: https://chatgpt.com/s/t_69875be666bc8191b634f9793895c379


